# Recurrent Neural Networks 

<img src="images/image8.png" height="400"/>

- Input at time step $t$: $x_t$
- Ouput at time step $t$: $y_t$

$$h_1=\tanh(W_h h_0 + W_x x_0)$$


$$y_1=Sotfmax(W_y h_1)$$

<img src="images/image9.png" height="400"/>

In [ ]:
# hidden state

# nn.RNN()

# input: [n_bacth, n_times, n_features]
